In [4]:
pip install oracledb

In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

# 1. DB에서 불러오기
import oracledb
conn = oracledb.connect(user="moa", password="moa", dsn="localhost:1521/xe")
df = pd.read_sql("SELECT * FROM mydata_transaction", con=conn)
conn.close()

# 2. 날짜 + 시간 결합 → datetime
df['datetime'] = pd.to_datetime(
    df['TRADE_DETAIL_DATE'].astype(str) + ' ' + df['TRADE_TIME'].astype(str),
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'
)

# 3. 파생 변수 생성
df['hour'] = df['datetime'].dt.hour                # 거래 시각 (0~23)
df['weekday'] = df['datetime'].dt.weekday          # 요일 (0=월, ..., 6=일)

# 4. 타겟 라벨 인코딩
df['label'] = df['TRANSACTION_TYPE'].map({'입금': 1, '출금': 0})

# 5. 범주형 컬럼 인코딩
le1 = LabelEncoder()
df['TRADE_TYPE_ENC'] = le1.fit_transform(df['TRADE_TYPE'])

le2 = LabelEncoder()
df['BANKBOOK_NOTE_ENC'] = le2.fit_transform(df['BANKBOOK_NOTE'])

# 6. 사용할 피처 선택
features = ['TRADE_AMOUNT', 'BALANCE_AFTER_TRADE', 'hour', 'weekday',
            'TRADE_TYPE_ENC', 'BANKBOOK_NOTE_ENC']

X = df[features]
y = df['label']

# 7. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train = X_train.replace([np.inf, -np.inf], np.nan)  # inf → NaN
X_train = X_train.fillna(0)

X_test = X_test.replace([np.inf, -np.inf], np.nan)  # inf → NaN
X_test = X_test.fillna(0)

y_train = y_train.replace([np.inf, -np.inf], np.nan)  # inf → NaN
y_train = y_train.fillna(0)

y_test = y_test.replace([np.inf, -np.inf], np.nan)  # inf → NaN
y_test = y_test.fillna(0)

# 8. 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 9. 예측 및 평가
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\n Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.3333333333333333

 Classification Report:
               precision    recall  f1-score   support

           0       0.33      1.00      0.50         1
           1       0.00      0.00      0.00         2

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3



e:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
e:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-def